In [3]:
pwd = '/nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/'

In [4]:
# Imports necessary utilities and modules

import ROOT as rt
import root_numpy as rtnp
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from collections import Counter 
import datetime
import pytz

# Graph/histo utilities from ROOT
# Contained within the delayed_jet_analyzer repository
import sys
sys.path.append(pwd+'lib')
from histo_utilities import create_TH1D, create_TH2D, create_TGraph, std_color_list

# Used for extracting the TTree structure from each datafile
import os
import uproot

# Sets display width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

donotdelete = []

In [16]:
fpath = {}
tree = {}
data_path = pwd+'data/'

fpath['test_file'] = data_path + 'jet_timing_studies_ntuple_metbb_ggh_ISR_mh125_mx50_pl1000_ev100000_test_1.root'
for k,v in fpath.items():
    print(str(datetime.datetime.now(pytz.timezone('US/Pacific'))))
    print(k, v)
    root_dir = uproot.open(v) 
    tree[k] = root_dir['ntuples']['llp']

2019-07-31 16:17:19.551369-07:00
test_file /nfshome/nasurijr/LLP_analysis/delayed_jet_analyzer/data/jet_timing_studies_ntuple_metbb_ggh_ISR_mh125_mx50_pl1000_ev100000_test_1.root


In [47]:
T = tree['test_file']
print(len(T))

5116


In [15]:
# print(T['nCsc'].array())

# # for ev in range(len(T_wh['nCsc'].array())):
# gParticleMotherId = T['gParticleMotherId'].array()
# gParticleId = T['gParticleId'].array()
# gParticleStatus = T['gParticleStatus'].array()
# # print(gParticleId)
# counter = 0
# # print(np.logical_and(np.logical_and(np.absolute(gParticleId) == 13, np.absolute(gParticleMotherId) == 24)))
# ele_sel = np.logical_and(np.logical_and(np.absolute(gParticleId) == 13, np.absolute(gParticleMotherId) == 24), gParticleStatus == 1 )
# mu_sel = np.logical_and(np.logical_and(np.absolute(gParticleId) == 11, np.absolute(gParticleMotherId) == 24), gParticleStatus == 1 )
# for ev in np.logical_or(ele_sel, mu_sel):
#     if ev.any():
# #         print(ev.any())
#         counter +=1
# print(counter)
# print(len(gParticleId))
# print(np.logical_and(np.logical_and(np.absolute(gParticleId) == 13, np.absolute(gParticleMotherId) == 24), gParticleStatus == 1 ))

In [38]:
# Variable Definitions
# Creates dictionaries for variables to be analyzed
# The dictionaries will contain the variable arrays for each datafile with a relevant key

# CSC
nCsc = {}
csc_z = {}
csc_x = {}
csc_y = {}
csc_eta = {}
csc_phi = {}
# Gen Level
gLLP_eta = {}
gLLP_r = {}
gLLP_decay = {}
eventNum_endcap = {}
lumiNum_endcap = {}
eventNum_barrel = {}
lumiNum_barrel = {}
eventNum_bkg = {}
lumiNum_bkg = {}
ncsc_barrel = {}
runNum = {}
gLLPr = {}

# Drift Tube (DT)
nDt = {}
dt_phi = {}
dt_eta = {}
dt_z = {}
dt_x = {}
dt_y = {}
# RPC
nRpc = {}
rpc_phi = {}
rpc_eta = {}
rpc_z = {}
rpc_x = {}
rpc_y = {}

In [39]:
data_trees = {'test': T}

In [40]:
# Event Selection

print('Start: ' + str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

# MET+bb Final State
if 'metbb' in fpath['test_file']:
    for species, arbor in data_trees.items():
        if 'vh' in fpath['test_file']:
                # Basic implementation: Loop through files and compile Boolean array with 1 muon/electron
                ele_sel = np.logical_and(np.logical_and(np.absolute(arbor['gParticleId'].array()) == 13, np.absolute(arbor['gParticleMotherId'].array()) == 24), arbor['gParticleStatus'].array() == 1)
                mu_sel = np.logical_and(np.logical_and(np.absolute(arbor['gParticleId'].array()) == 11, np.absolute(arbor['gParticleMotherId'].array()) == 24), arbor['gParticleStatus'].array() == 1)
    #             if not ((np.count_nonzero(mu) ==1 or np.count_nonzero(ele)==1)): continue
                sel_lepton = []
                time_counter = 0
                for ev in np.logical_or(ele_sel, mu_sel):
                    time_counter += 1
                    if (time_counter % 100000) == 0:
                        print(str(time_counter) + ': '+  str(datetime.datetime.now(pytz.timezone('US/Pacific'))))
                    if ev.any():
                        sel_lepton.append(True)
                    else:
                        sel_lepton.append(False)
                        
        if species in ('m50ct1m', 'm50ct10m', 'm975ct1m', 'm975ct10m', 'test'):
            gLLP_r[species] = np.sqrt((arbor['gLLP_decay_vertex_x'].array()[:,0])**2+(arbor['gLLP_decay_vertex_y'].array()[:,0])**2)
            gLLP_decay[species] = np.sqrt((arbor['gLLP_decay_vertex_x'].array()[:,0])**2+(arbor['gLLP_decay_vertex_y'].array()[:,0])**2 + (arbor['gLLP_decay_vertex_z'].array()[:,0])**2)

            # Endcap Signal Event Selection: Requires that the bb-decaying LLP decays within the endcap of the muon system        
            sel_csc = np.logical_and(np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) > 568 , np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) < 1100)
            sel_csc = np.logical_and(sel_csc, np.absolute(arbor['gLLP_eta'].array()[:,0])<2.4)
            sel_csc = np.logical_and(sel_csc, np.absolute(arbor['gLLP_eta'].array()[:,0])>0.9)
            sel_csc = np.logical_and(sel_csc, np.absolute(gLLP_r[species])<695.5)

            # Barrel Signal Event Selection: Requires that the bb-decaying LLP decays within the barrel of the muon system        
            sel_barrel = np.logical_and(np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) > -661 , np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) < 661)
            sel_barrel = np.logical_and(sel_barrel, np.absolute(gLLP_r[species])<738)
            sel_barrel = np.logical_and(sel_barrel, np.absolute(gLLP_r[species])>380)

            # Overlap Signal Event Selection: Requires that the bb-decaying LLP decays within the overlap of the barrel
            #                                 and endcap of the muon system
            sel_rpc = np.logical_and(np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) > 0 , np.absolute(arbor['gLLP_decay_vertex_z'].array()[:,0]) < 1100)
            sel_rpc = np.logical_and(sel_rpc, np.absolute(arbor['gLLP_eta'].array()[:,0])<1.6)
            sel_rpc = np.logical_and(sel_rpc, np.absolute(gLLP_r[species])<738)
            sel_rpc = np.logical_and(sel_rpc, np.absolute(gLLP_r[species])>275)

            # Total Muon System Signal Event Selection: Logical or of barrel and endcap gen-level restrictions
            sel_dis_signal = np.logical_or(sel_csc, sel_barrel)
        
            
            if 'vh' in fpath['test_file']:
                sel_csc = np.logical_and(sel_csc, sel_lepton)
                sel_barrel = np.logical_and(sel_barrel, sel_lepton)
                sel_rpc = np.logical_and(sel_rpc, sel_lepton)
#             print(len(sel_lepton))
#         break
        # Converts variable branches to numpy arrays with event selection
        # Signal conversions
#         if species in ('m50ct1m', 'm50ct10m', 'm975ct1m', 'm975ct10m'):
            # Endcap: CSCs
            nCsc[species] = arbor['nCsc'].array()[sel_csc]
            csc_z[species] = arbor['cscZ'].array()[sel_csc]
            csc_x[species] = arbor['cscX'].array()[sel_csc]
            csc_y[species] = arbor['cscY'].array()[sel_csc]
            csc_eta[species] = arbor['cscEta'].array()[sel_csc]
            csc_phi[species] = arbor['cscPhi'].array()[sel_csc]
            
            # Gen-level
            gLLP_eta[species] = arbor['gLLP_eta'].array()[:,0][sel_csc]
            eventNum_endcap[species] = arbor['eventNum'].array()[sel_csc]
            lumiNum_endcap[species] = arbor['lumiNum'].array()[sel_csc]
            runNum[species] = arbor['runNum'].array()[sel_csc]
            
            eventNum_barrel[species] = arbor['eventNum'].array()[sel_barrel]
            lumiNum_barrel[species] = arbor['lumiNum'].array()[sel_barrel]
            gLLPr[species] = gLLP_r[species][sel_barrel]
            ncsc_barrel = arbor['nCsc'].array()[sel_barrel]
            
            # Barrel: DTs
            nDt[species] = arbor['nDt'].array()[sel_barrel]
            dt_z[species] = arbor['dtZ'].array()[sel_barrel]
            dt_x[species] = arbor['dtX'].array()[sel_barrel]
            dt_y[species] = arbor['dtY'].array()[sel_barrel]
            dt_eta[species] = arbor['dtEta'].array()[sel_barrel]
            dt_phi[species] = arbor['dtPhi'].array()[sel_barrel]
            
            # Overlap: RPCs
            nRpc[species] = arbor['nRpc'].array()[sel_rpc]
            rpc_z[species] = arbor['rpcZ'].array()[sel_rpc]
            rpc_x[species] = arbor['rpcX'].array()[sel_rpc]
            rpc_y[species] = arbor['rpcY'].array()[sel_rpc]
            rpc_eta[species] = arbor['rpcEta'].array()[sel_rpc]
            rpc_phi[species] = arbor['rpcPhi'].array()[sel_rpc]
        
        # Background conversions
        if species in ('qcd', 'zeroBias', 'WJetsToLNu'):
            # Background event selection: Requires at least one jet per event to have pT > 50 GeV
            sel_bkg = [True if np.any(x > 50) else False for x in arbor['jetPt'].array()]
            if 'vh' in fpath['test_file']:
                sel_bkg = np.logical_and(sel_bkg, sel_lepton)
            
            # Endcap: CSCs
            nCsc[species] = arbor['nCsc'].array()[sel_bkg]
            csc_z[species] = arbor['cscZ'].array()[sel_bkg]
            csc_x[species] = arbor['cscX'].array()[sel_bkg]
            csc_y[species] = arbor['cscY'].array()[sel_bkg]
            csc_eta[species] = arbor['cscEta'].array()[sel_bkg]
            csc_phi[species] = arbor['cscPhi'].array()[sel_bkg]
            gLLP_eta[species] = arbor['gLLP_eta'].array()[:,0][sel_bkg]
            
            # Gen-level
            eventNum_bkg[species] = arbor['eventNum'].array()[sel_bkg]
            lumiNum_bkg[species] = arbor['lumiNum'].array()[sel_bkg]
            
            # Barrel: DTs
            nDt[species] = arbor['nDt'].array()[sel_bkg]
            dt_z[species] = arbor['dtZ'].array()[sel_bkg]
            dt_x[species] = arbor['dtX'].array()[sel_bkg]
            dt_y[species] = arbor['dtY'].array()[sel_bkg]
            dt_eta[species] = arbor['dtEta'].array()[sel_bkg]
            dt_phi[species] = arbor['dtPhi'].array()[sel_bkg]
            
            # Overlap: RPCs
            nRpc[species] = arbor['nRpc'].array()[sel_bkg]
            rpc_z[species] = arbor['rpcZ'].array()[sel_bkg]
            rpc_x[species] = arbor['rpcX'].array()[sel_bkg]
            rpc_y[species] = arbor['rpcY'].array()[sel_bkg]
            rpc_eta[species] = arbor['rpcEta'].array()[sel_bkg]
            rpc_phi[species] = arbor['rpcPhi'].array()[sel_bkg]
        
        print(species + ': ' +str(datetime.datetime.now(pytz.timezone('US/Pacific'))))

Start: 2019-07-31 16:27:23.715475-07:00
test: 2019-07-31 16:27:23.794358-07:00


In [44]:
# # Selection of Events for Event Display Purposes (Test area)
# print(len(eventNum_barrel['m50ct10m']))
# print(len(nDt_avg_ratio['m50ct10m'][0]))
index_ev = np.where(np.array(nCsc['test'])>10)
print('    Run '+ 'Lumi ' + 'Event ' + 'nCsc ')
print(np.dstack((runNum['test'][index_ev].astype(int), lumiNum_endcap['test'][index_ev].astype(int), eventNum_endcap['test'][index_ev].astype(int), nCsc['test'][index_ev])))

    Run Lumi Event nCsc 
[[[   1    1   44   26]
  [   1    1   72   58]
  [   1    1   75   53]
  [   1    2  122   17]
  [   1    2  178   61]
  [   1    2  195   63]
  [   1    3  237   92]
  [   1    3  275   16]
  [   1    4  350   57]
  [   1    5  483   71]
  [   1    6  541   42]
  [   1    6  588  115]
  [   1    7  644   12]
  [   1    7  699   19]
  [   1    9  853   29]
  [   1   92  188   69]
  [   1   94  321  123]
  [   1   95  447   44]
  [   1   96  544   15]
  [   1   97  651   72]
  [   1   98  757   75]
  [   1   99  808   52]
  [   1  100  955   20]
  [   1  100  964   40]
  [   1  100  983   93]
  [   1  100 1000  199]
  [   1  992  187   56]
  [   1  994  302  129]
  [   1  994  330  121]
  [   1  994  335   17]
  [   1  994  380  143]
  [   1  995  411   56]
  [   1  995  438   37]
  [   1  996  539   52]
  [   1  997  640  180]
  [   1  997  658   43]
  [   1  999  836   51]
  [   1  999  844   65]
  [   1 1000  939   16]
  [   1  102  160   44]
  [   1  104  3

In [46]:
print(len(runNum['test'][index_ev]))
print(len(lumiNum_endcap['test']))
print(len(nCsc['test']))

136
166
166
